(my-label)=
# Random numbers

In [ ]:
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})
# Bokeh libraries
from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider, WheelZoomTool, Legend, Span
from bokeh.plotting import ColumnDataSource, figure, output_file, show
from bokeh.palettes import Spectral11, Blues, viridis
from bokeh.io import output_notebook
output_notebook(hide_banner =  True)

All randoms number must be sampled from some distribution. One example of sampling a random number is throwing a dice where one can obtain any integer number between $1$ and $6$ with equal probabilities. As getting every number from $1$ to $6$ is equally likely, we say we sample the numbers uniformly or from a uniform distribution. We can easily create a 'digital' dice with python with the function `np.random.randint(1,7)`. The code belows throws the dice five times and prints the results:

In [ ]:
for i in range(5):
    dice_result = np.random.randint(1,7)
    print("Throw", i+1, ":", dice_result)

To show that out digital dice is fair, we can simply throw the dice many times ($120,000$ times) and plot a histogram for the occurrences of each number. We can do this with the following:

In [ ]:
all_results = []
total_throws = 120000
for i in range(total_throws):
    dice_result = np.random.randint(1,7)
    all_results.append(dice_result)
    
# Plot histogram of dice results
num_throws, edges = np.histogram(all_results, bins=range(1,8))
plt.bar(range(1,7), num_throws, alpha = 0.7, color='dodgerblue')
plt.xlabel('Dice result', fontsize = 12);
plt.ylabel('Number of throws', fontsize = 12);

Although with some minor differences, every number in the dice was obtained the same number of times (~$20,000$ times). Thus, it seems it is a fair dice. We can easily translate this histogram into a probability distribution by dividing the number of throws of each number by the total number of throws. Doing this division, we obtain the probability distribution.

In [ ]:
# Plot disitrbution of dice results
plt.bar(range(1,7), num_throws/total_throws, alpha = 0.7, color='dodgerblue')
plt.xlabel('Dice result', fontsize = 12);
plt.ylabel('Probability', fontsize = 12);

The height of each column is around $1/6\approx 0.1666$, and it corresponds to the probability of rolling any given  number in the dice. The probability of rolling any number from $1$ to $6$ is the sum of the probabilities of each number. This means $1/6 + 1/6 + 1/6 + 1/6 + 1/6 + 1/6 = 1$. This is a universal property of probabilities. The probabilities of all the possible events, should always sum to one.

## Uniform distribution

Random numbers are not limited to integer values like in a dice. For instance the function `np.random.uniform(xmin,xmax)` samples a random number uniformly in the interval $[xmin, xmax)$. We sample five values of this function with $x_\text{min} = 0$ and $x_\text{max}=100$:

In [ ]:
xmin = 0
xmax = 100
for i in range(5):
    random_number = np.random.uniform(xmin,xmax)
    print("Trial", i+1, ":", random_number)

We now show that the distribution of these numbers follows a uniform probability distribution. To do so, we separate our domain in $30$ bins to plot the histogram as before. To make sure it corresponds to a probability, we pass the argument `density=True` to the plotting function. This takes care of dividing the counts for each bin by the total number of trials automatically.

In [ ]:
all_numbers = []
total_trials = 100000
for i in range(total_trials):
    random_number = np.random.uniform(xmin,xmax)
    all_numbers.append(random_number)
    
# Plot histogram/distribution of random numbers
plt.hist(all_numbers, bins=30, density = True, alpha = 0.5, 
         edgecolor='black', lw = 0.3, color='dodgerblue', label="Distribution of sampled numbers")
plt.plot([xmin, xmin,xmax, xmax], [0, 1/(xmax - xmin),1/(xmax - xmin), 0], lw =2, 
         label="Uniform distribution", color='orange')
plt.title('Uniform distribution', fontsize = 12)
plt.xlabel('Random number', fontsize = 12);
plt.ylabel('Number of trials (normalized)', fontsize = 12);
plt.legend(fontsize = 12)

This distribution followed by the sampling numbers corresponds to a uniform distribution since every real number between $x_\text{min}$ and $x_\text{max}$ is sampled uniformly. Also note the area covered by the histogram is around $0.01 \times 100 = 1$, in the same way as the probabilities of all the numbers in the dice sum to one. In general, we can represent the uniform distribution as a constant line, as plotted in the previous figure:

$$
\rho_\text{uniform} = 1/(x_\text{max} - x_\text{min})
$$ 

with $x$ between $x_\text{min}$ and $x_\text{max}$, such that the area covered by the distribution, or more precisely, the integral of the distribution is equal to 1:

$$
\int_{x_\text{min}}^{x_\text{max}}\rho_\text{uniform}dx = \frac{x_\text{max} - x_\text{min}}{x_\text{max} - x_\text{min}} = 1
$$

## Normal distribution

Another famous distribution is the normal or Gaussian distribution. This distribution has the following form

$$
\rho_\text{normal}(x)=\frac{1}{\sigma\sqrt{2\pi}}\exp \Big( -\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2 \Big),
$$

where $\mu$ corresponds to the mean and $\sigma$ to the standard deviation. It can be shown its integral from $-\infty$ to $\infty$ is always equal to one. Plotting this function, we obtain the shape of the normal distirbution. Try changing the mean and the variance to see how the distribution changes.

In [ ]:
# Bokeh interactive plot code for normal distribution
x = np.linspace(-4,4,80)
y = 1/(np.sqrt(2 * np.pi)) * np.exp(-0.5*x**2)
source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(x_range=(-4, 4), y_range=(0, 1), plot_width=400, plot_height=300, tools="pan,wheel_zoom,reset")
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6, color='orange')
plot.title.text = 'Normal distribution'
plot.title.text_font_size = '12pt'
plot.toolbar.logo = None

mu_slider = Slider(start=-3.5, end=3.5, value=0, step=.1, title="Mean")
sigma_slider = Slider(start=0.1, end=3, value=1, step=.1, title="Standard deviation")

callback = CustomJS(args=dict(source=source, mu=mu_slider, sigma=sigma_slider),
                    code="""
    const data = source.data;
    const mean = mu.value;
    const stddev = sigma.value;
    const x = data['x']
    const y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = 1/(stddev * Math.sqrt(2 * Math.PI)) * Math.exp(-0.5*((x[i]-mean)/stddev)**2)
    }
    source.change.emit();
""")

mu_slider.js_on_change('value', callback)
sigma_slider.js_on_change('value', callback)

layout = row(
    plot,
    column(mu_slider, sigma_slider),
)

show(layout)

This distirbution implies that numbers closer to the mean value $\mu$ are much more likely to be sampled than those far away from the mean value. In python one can easily draw random numbers following this distribution by using the `np.random.normal(mu,sigma)` function:

In [ ]:
mu = 0
sigma = 1
for i in range(5):
    random_number = np.random.normal(mu,sigma)
    print("Trial", i+1, ":", random_number)

We denote a random number sampled from this distribution as $\mathcal{N}(\mu,\sigma)$, where $\sigma^2$ is the variance and corresponds to the standard deviation squared. It is easy to test the sampled numbers actually follow a normal distribution. Lets sample a large number of values with this function and plot their distribution as before:

In [ ]:
# Sample random values from a standard normal distibution
values = []
num_samples = 12000
for i in range(num_samples):
    r = np.random.normal(mu,sigma)
    values.append(r)

# Plot histogram of sampled numbers (normalized frequency distribution)
plt.hist(values,30, density=True, alpha =0.5, color='dodgerblue', edgecolor='black', lw=0.3,
         label="Histogram of sampled numbers distribution")
    
# Plot probability distribution    
x = np.linspace(-4,4,60)
y = 1/(np.sqrt(2 * np.pi)) * np.exp(-0.5*x**2)
plt.plot(x,y, lw =2, color='orange', label="Standard normal distribution")
plt.ylim([0,0.55])
plt.legend(fontsize = 12);

Now we know how to sample random numbers from uniform and normal/Gaussian distributions. 